## RandomForest 
Scaling version 1 from 02_feature_engineering
used dummies for pure categoricals, kept numericals and scaled them using StandardScaler.

#### First Exploration
Try a random forest and see how it performs!

In [ ]:
#initial imports 
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Optional
import seaborn as sns
import matplotlib.pyplot as plt

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Get the absolute path of the current file/notebook
# If using Jupyter, use Path.cwd(). If using a .py script, use Path(__file__).parent
curr_dir = Path.cwd()

# Calculate the project root (adjust '.parent' count as needed)
# If your notebook is in 'project/notebooks/', the root is 1 level up
project_root = curr_dir.parent.parent 

# Add project root to system path so Python can find 'utils'
sys.path.append(str(project_root))

print(f"Project Root added to path: {project_root}")

from utils.feature_engineer_df import build_features 

#for the scaling and encoding 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#cleanup 

pd.set_option('display.max_columns', None)

In [ ]:
#get your data from our utils
build_features(
    input_path=Path.cwd().resolve().parents[1] / "data" / "cleaned" / "kickstarter_cleaned.csv",
    output_path=Path.cwd().resolve().parents[1] / "data" / "feature" / "kickstarter_featured.csv",
    raw_path=Path.cwd().resolve().parents[1] / "data" / "raw" / "ks-projects-201801.csv",
    logger=logger
)

# Load Files as DataFrames
BASE_DIR = Path.cwd().resolve().parents[1]
data_file = BASE_DIR / "data" / "feature" / "kickstarter_featured.csv"

filepath = Path(data_file)

df = pd.read_csv(filepath, encoding='latin-1', low_memory=False)

In [ ]:
#list of columns to "hard drop" from feature engineering dataframe
columns_to_drop = ['id', #irrelevant
                   'main_category', #substituted in a satisfactory way
                   'deadline', 'launched', #created new categories 
                   'backers', 'usd_pledged_real', 'usd_pledged_bins', 'backers_per_pledged', 'backer_pledged_bins', 'pledged_per_category', #everything to do with "future information"
                   'launched_year', 'deadline_year', #info about the past and not seasonal
                   ]
# drop them
dfc = df.drop(columns=columns_to_drop)

Get dummies for pure categoricals:

In [ ]:
#season launched, dropping first as it's multicollinear
df_sl = pd.get_dummies(df['launch_season'], prefix = 'sl_', drop_first=True, dtype=int)

#season deadline, dropping first
df_sd = pd.get_dummies(df['deadline_season'], prefix = 'sd_', drop_first=True, dtype=int)

#main category_grouped, dropping first
df_cat = pd.get_dummies(df['main_category_grouped'], prefix = 'cat_', drop_first=True, dtype=int)

#continent, dropping first
df_co = pd.get_dummies(df['continent'], prefix = 'co_', drop_first=True, dtype=int)

#put everything back together again: 
dff = pd.concat([dfc, df_sl, df_sd, df_cat, df_co], axis=1)


In [ ]:
#let's soft-drop everything we just encoded
#commented out all the keepers 
columns_to_softdrop = ['country', #we kept it for comparison
                       # 'usd_goal_real', #right now I want to try scaling actual values 
                       #'duration_days', #I want to scale these and drop the bins instead for now 
                       # 'target', (obviously)
                       'main_category_grouped', 'continent', #after creating dummies, get rid of these!
                        'launched_month', 'deadline_month', #because we have season but might want to look closer
                        'usd_goal_bins', #using category_goal_percentile (those two are redundant)
                        #'goal_per_category', #it's a polynomial feature - not independent but that's probably ok
                       'category_goal_percentile', #it's an orinal bin so keeping 'goal per category' instead
                       'duration_bins', #want to use actual values instead, using duration_days
                       'launch_season', 'deadline_season', #gotten dummies 
                       #'duration_bins_coded', #dropped the whole encoding code 
                       ]
# keeping the already dummied ones obviously 

In [ ]:
df_to_scale = dff.drop(columns=columns_to_softdrop, axis=1)
display(df_to_scale.columns)

Scale the remaining numerical columns 

In [ ]:
#first, create our dfs 
X = df_to_scale.drop(columns=['target'])
y = df_to_scale['target']
#get train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)
print("Df before", df_to_scale.shape)
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

In [ ]:
#get the columns to standardise 
col_scale = ['usd_goal_real',
             'duration_days',
             'goal_per_category',
             ]

#instantiate
scaler = StandardScaler()
#scale 
X_train_scaled = scaler.fit_transform(X_train[col_scale])
X_test_scaled = scaler.fit_transform(X_test[col_scale])
#make it a df again
X_train_scaled = pd.DataFrame(
    X_train_scaled,
    columns=col_scale,
    index=X_train.index
)

X_test_scaled = pd.DataFrame(
    X_test_scaled,
    columns=col_scale,
    index=X_test.index
)

In [ ]:
#drop the original axes again
X_train = X_train.drop(col_scale, axis=1)
X_test = X_test.drop(col_scale, axis=1)
#and check if everything's still in order 
X_train.index.equals(X_train_scaled.index)
X_test.index.equals(X_test_scaled.index)

In [ ]:
# put it back together again 
X_train_sp = pd.concat([X_train_scaled, X_train ], axis=1)
X_test_sp = pd.concat([X_test_scaled, X_test], axis=1)
#and check
print("Dff shape", dff.shape)
print("X_train shape", X_train_scaled.shape)
print("X_test shape", X_test_scaled.shape)
print("X_train shape after scaling", X_train_sp.shape)
print("X_test shape after scaling", X_test_sp.shape)
print("train split head:")
display(X_train_sp.head())
print("test split head:")
display(X_test_sp.head())

### Get a baseline first

In [ ]:
corr = df_to_scale.corr()
sns.heatmap(data=corr, annot=True, linewidth=0.5, cmap='coolwarm')

## Random Forest 
using the train-test-split: 
* X_train_sp
* X_test_sp
* y_train
* y_test

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix

In [ ]:
#get features for feature importance 
features = list(X_train_sp.columns)
#create a model
model = RandomForestClassifier(n_estimators = 50, #start low
                               random_state = 42, #just any
                               max_features=8, #manually set it to 1/2 dataset 
                               n_jobs=-1, #use whole CPU
                               )
#fit the model
model.fit(X_train_sp, y_train)

In [ ]:
#first description of the model 
n_nodes = []
max_depths = []
#create stats 
for tree in model.estimators_:
  n_nodes.append(tree.tree_.node_count) #ask the tree how many nodes it has
  max_depths.append(tree.tree_.max_depth)
#get averages  
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
#check the original
train_predictions = model.predict(X_train_sp)
train_probabilites = model.predict_proba(X_train_sp)[:, 1] #get the positives
#check with test: 
test_predictions = model.predict(X_test_sp)
test_probabilities = model.predict_proba(X_test_sp)[:, 1]

print(f'Train ROC AUC Score: {roc_auc_score(y_train, train_probabilites)}')
print(f'Test ROC AUC Score: {roc_auc_score(y_test, test_probabilities)}')
print(f"Baseline ROC AUC calling everyone successful: {roc_auc_score(y_test, [1 for _ in range(len(y_test))])}")

In [ ]:
matrix = confusion_matrix(y_test, test_predictions)
print(matrix)

In [ ]:
fi_model = pd.DataFrame({'feature': features,
                         'importance': model.feature_importances_}).sort_values('importance', ascending=False)
fi_model.head()

### one shot at optimization

In [ ]:
#hyperparameter grid 
param_grid = {
    'n_estimators': [10, 20, 40, 50, 70, 90, 120], #just because I can, fibonacci-style guess
    'max_depth': [None] + list(np.arange(3, 21).astype(int)),
    'max_features': ['auto', 'sqrt', None, 8], 
    'max_leaf_nodes': [20, 100, 1000, 3000, 8000, 20000, 60000, 100000],
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

#get an instance to use for random search:
estimator = RandomForestClassifier(random_state = 42)
#create random search model 
rs = RandomizedSearchCV(estimator,
                        param_grid, 
                        n_jobs = -1,
                        scoring = 'roc_auc',
                        cv = 5,
                        n_iter = 10,
                        verbose = 4, 
                        random_state = 42)

# fit it 
rs.fit(X_train_sp, y_train)

In [ ]:
rs.best_params_

### go for best model

In [ ]:
best_model = rs.best_estimator_

In [ ]:
best_train_predictions = best_model.predict(X_train_sp)
best_train_probas = best_model.predict_proba(X_train_sp)[:, 1]
#and again 
best_test_predictions = best_model.predict(X_test_sp)
best_test_probas = best_model.predict_proba(X_test_sp)[:, 1]

In [ ]:
#now after optimization
b_n_nodes = []
b_max_depths = []

for tree in best_model.estimators_:
    b_n_nodes.append(tree.tree_.node_count)
    b_max_depths.append(tree.tree_.max_depth)

print(f'Average number of nodes {int(np.mean(b_n_nodes))}')
print(f'Average maximum depth {int(np.mean(b_max_depths))}')    

In [ ]:
new_cm = confusion_matrix(y_test, best_test_predictions)
print(matrix)

In [ ]:
print(f'Train ROC AUC Score: {roc_auc_score(y_train, best_train_probas)}')
print(f'Test ROC AUC Score: {roc_auc_score(y_test, best_test_probas)}')
print(f"Baseline ROC AUC calling everyone successful: {roc_auc_score(y_test, [1 for _ in range(len(y_test))])}")

In [ ]:
fi_model = pd.DataFrame({'feature': features,
                         'importance': best_model.feature_importances_}).sort_values('importance', ascending=False)
fi_model.head()